In [ ]:
import logging

import cm1.input.sounding
import matplotlib.pyplot as plt
import pandas as pd
from cm1.skewt import skewt
from cm1.utils import TMPDIR
from metpy.units import units

# Configure logging
logging.basicConfig(
    level=logging.WARNING, format="%(asctime)s - %(levelname)s: %(message)s", force=True
)

### Predefined CM1 input soundings

In [ ]:
dss = [
    cm1.input.sounding.trier(),
    cm1.input.sounding.jordan_allmean(),
    cm1.input.sounding.jordan_hurricane(),
    cm1.input.sounding.rotunno_emanuel(),
    cm1.input.sounding.dunion_MT(),
    cm1.input.sounding.bryan_morrison(),
    cm1.input.sounding.seabreeze_test(),
]
fig, axes = plt.subplots(
    ncols=2,
    sharex=True,
    sharey=True,
    figsize=(10, 16),
    gridspec_kw={"hspace": 0.4},
)
# Hide the axes for all subplots
for ax in axes.flat:
    ax.axis("off")

for i, ds in enumerate(dss):
    # Use subplot because skewt() uses metpy.plots.SkewT, which uses it.
    logging.info(ds.attrs["case"])
    skew = skewt(ds, fig=fig, subplot=(4, 2, i + 1))
    skew.ax.set_title(
        ds.attrs["case"] + skew.ax.get_title(), fontsize="x-small"
    )  # Set title to sounding case

plt.tight_layout()

In [ ]:
valid_time = pd.to_datetime("20240525")
lon = -95 * units.degree_E
lat = 33 * units.degree_N
ds_aws = cm1.input.sounding.era5_aws(valid_time, lat, lon)
ds_aws

### Given a latitude and longitude, select a 3x3 grid (nearest ERA5 grid point + 8 neighbors)

In [ ]:
# ds0 = cm1.input.era5.model_level(valid_time).load()
ds0 = cm1.input.era5.aws(valid_time)

# Use scalar values for lat/lon to select the central grid point
sel = cm1.input.era5.nearest_grid_block_sel(ds0, lat=lat, lon=lon)
ds0 = ds0.sel(sel)

fig, axes = plt.subplots(ncols=3, nrows=3, sharex=True, sharey=True, figsize=(15, 14))
# Hide the axes for all subplots
for i, ax in enumerate(axes.flat):
    ax.axis("off")
    ds = ds0.stack(i=("latitude", "longitude")).isel(i=i)
    skew = skewt(ds, fig=fig, subplot=(3, 3, i + 1))
    skew.ax.set_title(f"{skew.ax.get_title()}", fontsize="x-small")
    ofile = TMPDIR / f"{skew.ax.get_title().split('\n')[0].replace(' ','_')}.txt"
    with open(ofile, "w") as fh:
        fh.write(cm1.input.sounding.to_txt(ds))
        logging.warning(ofile)

In [ ]:
import metpy.calc as mpcalc

ds = ds0
# Attach MetPy parsing (to interpret units and coordinates)
ds = ds.metpy.parse_cf()

# Get wind components (assumes they are in m/s and with coordinates parsed)
u = ds["U"].metpy.sel(level=850 * units.hPa)  # or any other level
v = ds["V"].metpy.sel(level=850 * units.hPa)

# Grid spacing (assumes 1D lat/lon)
dx, dy = mpcalc.lat_lon_grid_deltas(ds["longitude"], ds["latitude"])

# Compute divergence (returns units of 1/s)
div = mpcalc.divergence(u, v, dx=dx, dy=dy)
# `div` is a 2D array (latitude x longitude) at the selected level
ds["divergence"] = (("latitude", "longitude"), div.data)
ds["divergence"].plot()

In [ ]:
cm1.input.era5.pressure_level(valid_time).load()

In [ ]:
ds0.LSM.plot()

In [ ]:
from io import StringIO

s = cm1.input.sounding.to_txt(cm1.input.sounding.trier())
print(s)
pd.read_csv(StringIO(s), skiprows=1, sep=" ", names=["Z", "theta", "Q", "U", "V"])

## Show difference between pressure-level and model level soundings
* fewer pressure levels than model levels
* winds at surface (10u and 10v) not in pressure-level sounding

In [ ]:
fig, axes = plt.subplots(ncols=2, sharey=True, figsize=(14, 7))
# Hide the axes for all subplots
for ax in axes.flat:
    ax.axis("off")
skew = skewt(ds_aws, fig=fig, subplot=(1, 2, 1))
skew.ax.set_title(f"ERA5 AWS pressure level  {skew.ax.get_title()}", fontsize="x-small")
skew = skewt(ds0.isel(latitude=1, longitude=1), fig=fig, subplot=(1, 2, 2))
skew.ax.set_title(f"model level  {skew.ax.get_title()}", fontsize="x-small")

In [ ]:
cm1.input.sounding.to_txt(ds_aws)

In [ ]:
ds = cm1.input.sounding.era5_pressure_level(valid_time, lat, lon)
ds

In [ ]:
skewt(ds)